# How we create patches from tiles

In [1]:
import torch
from darts_segmentation.utils import patch_coords
import matplotlib.patches as mpl_patches
import matplotlib.pyplot as plt

In [4]:
# Example parameters
h, w = 30, 30
patch_size = 8
overlap = 3

# Create an example tile (already as torch tensor)
tensor_tiles = torch.rand((3, 1, h, w))

In [ ]:
# Visualize the patching
fig, ax = plt.subplots(1, 1, figsize=(20, 20))
ax.imshow(tensor_tiles[0, 0])
colors = ["red", "orange", "grey", "brown", "yellow", "purple", "teal"]
for i, (y, x, patch_idx_y, patch_idx_x) in enumerate(patch_coords(h, w, patch_size, overlap)):
    c = colors[i % len(colors)]
    rect = mpl_patches.Rectangle(
        (x - 0.5, y - 0.5), width=patch_size, height=patch_size, linewidth=3, edgecolor=c, facecolor=c, alpha=0.5
    )
    ax.add_patch(rect)
    ax.text(x, y, f"{i}: {patch_idx_x}-{patch_idx_y} ({x}-{y})", bbox={"facecolor": "white"})